# Tableau Project - Videogame Co.

## Where do we open our new office?

### Instrucciones
#### Somos una compañía de Videogames de 50 empleados. Tenemos que abrir una oficina nueva, pero, ¿dónde?
#### Contamos con 20 desarrolladores, 20 UX/UI y 10 ejecutivos.
#### Tiene que haber alrededor:
#### 1-. Compañías de Software
#### 2-. Buen ratio entre grades compañías y start-ups
#### 3-. Empresas de diseño, diseño web, relacionado con UX
#### 4-. Restaurantes para que los ejecutivos puedan reunirse y hacer negocios 
#### 5-. Evitar compañías muy viejas alrededor

### Variables interesantes que mirar
#### Total money raised (inversiones), competitors, latitud y longitud de oficinas, twitter (influencia de la empresa, API tweety), cathegory code

In [133]:
import re
import pandas as pd
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

def convertOffice(companies):
    return {
        "name": companies["name"],
        "cuisine": companies["cuisine"],
        "position": toGeoJSON(companies["address"]["coord"])
    }

cursor = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor))


df = pd.DataFrame(geoRestaurants)
df.head()

In [141]:
# Para evitar compañías viejas alrededor, vamos a considerar qué es una compañía vieja, respondiendo a: ¿En qué año se fundaron
# las compañías más importantes de tecnología hoy en día? Google en el 1998, Intel 1968, Microsoft 1975, Apple 1976
# Tenemos en cuenta las compañías más importantes, ordenadas por valuation amount descending.

#Primero preguntamos a mongo por 1000 compañías de Software para sacar su lat y long.
#Software Co

cursor1 = db.companies.find({"$and": [{"category_code": 'software'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

software_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor1)))

software = pd.DataFrame(software_co)

In [135]:
#También preguntamos a mongo por 500 compañías de Mobile para sacar su lat y long, nos interesa tener empresas de movil cerca para sacar videojuegos en el movil.
#Mobile Co

cursor2 = db.companies.find({"$and": [{"category_code": 'mobile'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(500)

mobile_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor2)))

mobile = pd.DataFrame(mobile_co)

In [136]:
#También nos interesa saber dónde está la competencia, ya que significa que hay mercado en esa zona. 
#Preguntamos a mongo por 1000 compañías de VideoGames para sacar su lat y long.
#VideoGames Co

cursor3 = db.companies.find({"$and": [{"category_code": 'games_video'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

gamesvideo_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor3)))

gamesvideo = pd.DataFrame(gamesvideo_co)

In [113]:
#Preguntamos a mongo por 1000 compañías de web para sacar su lat y long.
#Web Co

cursor4 = db.companies.find({"$and": [{"category_code": 'web'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

web_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor4)))

web = pd.DataFrame(web_co)


In [142]:
cos = software + mobile + gamesvideo + web
cos.columns = ['Company', 'Address']
print(len(cos))


1000


In [143]:
cos = cos.dropna(subset=['Address'], axis=0)
print(len(cos))
cos.head(11)

500


,Company,Address
0,BMC SoftwareMixiGREEGroupon,"[{'description': '', 'address1': '2101 City We..."
1,RackspaceTelenavTudouTencent,"[{'description': 'HQ', 'address1': '5000 Walze..."
2,Jive SoftwareMeru NetworksDeNATripAdvisor,"[{'description': 'Palo Alto HQ', 'address1': '..."
3,QlikTechHelioDeNAOpenTable,"[{'description': 'World Headquarters', 'addres..."
4,Cornerstone OnDemandJingle NetworksJoostWix,"[{'description': 'Headquarters', 'address1': '..."
5,SolarWindsJajahBabelgumWetpaint,"[{'description': 'HQ', 'address1': '3711 South..."
6,MarketoGrandCentralSparterPostini,"[{'description': 'Marketo Headquarters', 'addr..."
7,DemandTecZenZuiKyteGeni,"[{'description': 'Corporate Headquarters', 'ad..."
8,PironetiSkootVeohGizmoz,"[{'description': None, 'address1': None, 'addr..."
9,PironetEQOStickamFox Interactive Media,"[{'description': None, 'address1': '', 'addres..."


In [223]:
#Limpiar columna Address para dejar coordenadas

In [192]:
# cos.iloc[0]['Address']
# cos.iloc[0]['Address'][0]['latitude']

In [219]:
l = []
names = []
for i in range(500):
    norm = json_normalize(cos.iloc[i]['Address'])
    l.append(norm)
    
    name = cos.iloc[i]['Company']
    
    for n in range(len(norm)): 
        names.append(name)

print(l)


[                                          address1  \
0                              2101 City West Blvd   
1  Sumitomo Realty & Development Harajuku Building   
2      Roppongi-Hills Mori Tower, 6-10-1, Roppongi   
3                               600 W. Chicago Ave   
4                                  3101 Park Blvd.   
5                                    385 Grove St.   
6                             505 5th Avenue South   

                        address2           city country_code  \
0                                       Houston          USA   
1  2-34-17 Jingumae, Shibuya-ku,          Tokyo          JPN   
2                      Minato-ku          Tokyo          JPN   
3                      Suite 400        Chicago          USA   
4                                     Palo Alto          USA   
5                                 San Francisco          USA   
6                      Suite 615        Seattle          USA   

            description   latitude  longitude state_c

In [221]:
df_500 = pd.concat(l)
df_500.head(30)

,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code
0,2101 City West Blvd,,Houston,USA,,29.780549,-95.345792,TX,77042
1,Sumitomo Realty & Development Harajuku Building,"2-34-17 Jingumae, Shibuya-ku,",Tokyo,JPN,Headquarter,NaN,NaN,None,150-0001
2,"Roppongi-Hills Mori Tower, 6-10-1, Roppongi",Minato-ku,Tokyo,JPN,Headquarters,NaN,NaN,None,106-0032
3,600 W. Chicago Ave,Suite 400,Chicago,USA,HQ,41.896516,-87.643622,IL,60654
4,3101 Park Blvd.,,Palo Alto,USA,Palo Alto Office,NaN,NaN,CA,94086
5,385 Grove St.,,San Francisco,USA,San Francisco Office,NaN,NaN,CA,94102
6,505 5th Avenue South,Suite 615,Seattle,USA,Seattle Office,NaN,NaN,WA,98104
0,5000 Walzem Road,Suite 100,San Antonio,USA,HQ,29.527664,-98.562993,CA,78218
1,Millington Road,,London,GBR,London Office,NaN,NaN,None,UB3 4AZ
2,620 Folsom St.,#100,San Francisco,USA,San Francisco Office,NaN,NaN,CA,94107


In [222]:
df_500['Company'] = names
df_500.head(30)

,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code,Company
0,2101 City West Blvd,,Houston,USA,,29.780549,-95.345792,TX,77042,BMC SoftwareMixiGREEGroupon
1,Sumitomo Realty & Development Harajuku Building,"2-34-17 Jingumae, Shibuya-ku,",Tokyo,JPN,Headquarter,NaN,NaN,None,150-0001,BMC SoftwareMixiGREEGroupon
2,"Roppongi-Hills Mori Tower, 6-10-1, Roppongi",Minato-ku,Tokyo,JPN,Headquarters,NaN,NaN,None,106-0032,BMC SoftwareMixiGREEGroupon
3,600 W. Chicago Ave,Suite 400,Chicago,USA,HQ,41.896516,-87.643622,IL,60654,BMC SoftwareMixiGREEGroupon
4,3101 Park Blvd.,,Palo Alto,USA,Palo Alto Office,NaN,NaN,CA,94086,BMC SoftwareMixiGREEGroupon
5,385 Grove St.,,San Francisco,USA,San Francisco Office,NaN,NaN,CA,94102,BMC SoftwareMixiGREEGroupon
6,505 5th Avenue South,Suite 615,Seattle,USA,Seattle Office,NaN,NaN,WA,98104,BMC SoftwareMixiGREEGroupon
0,5000 Walzem Road,Suite 100,San Antonio,USA,HQ,29.527664,-98.562993,CA,78218,RackspaceTelenavTudouTencent
1,Millington Road,,London,GBR,London Office,NaN,NaN,None,UB3 4AZ,RackspaceTelenavTudouTencent
2,620 Folsom St.,#100,San Francisco,USA,San Francisco Office,NaN,NaN,CA,94107,RackspaceTelenavTudouTencent


In [194]:
lats = []
lons = []
for y in l:
    lat, lon = l.get('latitude'), l.get('longitude')
    # print(lat, lon)
    lats.append(lat)
    lons.append(lon)

AttributeError: 'list' object has no attribute 'get'

In [202]:
la = []
lo = []

for i in range(5):
    lax, lox = cos.iloc[i]['Address'][i]['latitude'], cos.iloc[i]['Address'][i]['longitude']
    la.append(lax)
    lo.append(lox)

print(la)
print(lo)

# cos.iloc[0]['Address']
# cos.iloc[0]['Address'][0]['latitude']

[29.7805488, None, None, None, None]
[-95.3457921, None, None, None, None]


In [170]:
lats = []
lons = []
for y in listadir:
    for q in y:
        lat, lon = q.get('latitude'), q.get('longitude')
        # print(lat, lon)
        lats.append(lat)
        lons.append(lon)
            
     #No me deja iterar sobre un diccionario, solo me deja print it  (esto de abajo no me lo deja hacer)      
            
    #latlong.append(y.get('latitude'))
    #latlong.append(y.get('longitude'))
    
# for y in listadir:
  #  for q in y:
      #  q.values()
        #del q['address1']
        #q.pop('address1', None)
        
#print(q)

cos['latitude'] = lats
cos['longitude'] = lons

cos.head(20)

ValueError: Length of values does not match length of index

In [149]:
for y in listadir:
    for q in y:
        for n in q:
            json_normalize(n)


AttributeError: 'str' object has no attribute 'values'

In [ ]:
from pandas.io.json import json_normalize

for i in range(100):
    json_normalize(y[i])
    
 

In [165]:
len(y)   

7

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

def convertRestaurant(restaurant):
    return {
        "company": cos["Company"],
        "position": toGeoJSON(cos["Address"]["coord"])
    }

cursor5 = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor5))


df = pd.DataFrame(geoRestaurants)
df.head()

In [215]:
# json_normalize(cos.iloc[0])
result = json_normalize(cos, 'Company', ['Address', 'latitude'])
result.head()

TypeError: string indices must be integers